# Self-Contained Gradio UI and GPT Integration Notebook

This notebook contains all the necessary code to set up a Gradio UI for the AI Financial Advisor and integrate it with GPT for generating financial insights. No external .py files are required.

## Setup

First, let's import the necessary libraries and set up our environment.

In [7]:
%pip install python-dotenv pandas gspread oauth2client openai==0.28.0 pdfkit gradio

In [2]:
import os
import pandas as pd
import openai
import gradio as gr
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set
if not openai.api_key:
    raise ValueError("OpenAI API key is not set. Please check your .env file.")

/Users/vincentkoc/GIT/_Perso/bitesize-notebook-financial-advisor/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Financial Data

Let's define functions to load our financial data from either a CSV file or Google Sheets.

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def load_from_google_sheets():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"), scope)
    client = gspread.authorize(creds)

    sheet = client.open_by_url(os.getenv("DUMMY_DATA_SPREADSHEET")).sheet1
    data = sheet.get_all_records()
    return pd.DataFrame(data)

def load_from_csv():
    csv_path = '../data/data.csv'  # Assuming the CSV file is in the same directory as the notebook
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"The file {csv_path} does not exist.")
    return pd.read_csv(csv_path)

def load_data():
    if os.getenv("USE_GOOGLE_SHEETS", "false").lower() == "true":
        return load_from_google_sheets()
    else:
        return load_from_csv()

# Load the financial data
df = load_data()

print("Data loaded successfully. Shape:", df.shape)

Data loaded successfully. Shape: (121, 5)


## Define Financial Analysis Functions

Now, let's define the functions for generating financial summaries and personalized advice.

In [4]:
def generate_financial_summary(df):
    total_spent = df[df['Income/Expense'] == 'Expense']['Amount'].sum()
    total_income = df[df['Income/Expense'] == 'Income']['Amount'].sum()
    category_spending = df[df['Income/Expense'] == 'Expense'].groupby('Category')['Amount'].sum()

    completion = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial assistant. Generate a concise financial summary based on the provided data."},
            {"role": "user", "content": f"Generate a financial summary based on the following data:\n"
                                        f"Total Income: ${total_income:.2f}\n"
                                        f"Total Expenses: ${total_spent:.2f}\n"
                                        f"Spending by category: {category_spending.to_dict()}"}
        ]
    )
    return completion.choices[0].message.content.strip()

def generate_personalized_advice(df, age=None, lifestyle=None, hobbies=None):
    transactions = df.to_string(index=False)

    prompt = f"""
    Based on the following transaction data:

    {transactions}

    And considering the following personal information:
    Age: {age}
    Lifestyle: {lifestyle}
    Hobbies: {hobbies}

    Please provide personalized financial advice. Include suggestions for budgeting, saving, and potential areas for improvement.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial advisor."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

## Define Advisor Interface

Now, let's define the advisor interface function that will handle user requests for financial summaries and advice.

In [5]:
def advisor_interface(request_type, age, location, hobbies):
    if request_type == "Summary":
        return generate_financial_summary(df)
    elif request_type == "Advice":
        return generate_personalized_advice(df, age=age, lifestyle=location, hobbies=hobbies)
    else:
        return "Invalid request type. Please choose 'Summary' or 'Advice'."

# Test the function
print("Testing advisor interface...")
print(advisor_interface("Summary", 30, "Urban", "Reading"))
print("---")
print(advisor_interface("Advice", 30, "Urban", "Reading"))

Testing advisor interface...
### Financial Summary

**Total Income:** $39,952.00  
**Total Expenses:** $22,590.21  
**Net Income:** $17,361.79  

#### Spending Breakdown by Category:
- **Accommodation:** $9,250.00
- **Charity:** $900.00
- **Dining Out:** $1,081.50
- **Electronics:** $3,500.00
- **Entertainment:** $581.96
- **Expense:** $200.00
- **Fitness:** $160.00
- **Groceries:** $2,292.00
- **Insurance:** $600.00
- **Miscellaneous:** $370.00
- **Shopping:** $1,205.00
- **Transportation:** $693.00
- **Travel:** $950.00
- **Utilities:** $806.75

#### Summary:
- Total income has significantly exceeded total expenses, resulting in a healthy net income of $17,361.79. The majority of expenses were allocated to accommodation, electronics, and groceries. Overall, financial management appears to be positive, with substantial savings and careful monitoring of essential expenditures.
---
Based on your transaction data and personal information, here’s a tailored financial analysis and recommen

## Create Gradio Interface

Now, let's create the Gradio interface for our AI Financial Advisor.

In [6]:
def launch_gradio_ui():
    iface = gr.Interface(
        fn=advisor_interface,
        inputs=[
            gr.Dropdown(
                choices=["Summary", "Advice"],
                label="Select summary or advice"
            ),
            gr.Number(label="Age"),
            gr.Textbox(label="Lifestyle (optional)", placeholder="e.g., Urban, Rural, Suburban"),
            gr.Textbox(label="Hobbies (optional)", placeholder="e.g., Reading, Sports, Travel")
        ],
        outputs="text",
        title="AI Financial Advisor",
        description="Get a financial summary or personalized advice based on your transaction history."
    )
    return iface

# Create and launch the Gradio interface
iface = launch_gradio_ui()
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## Interacting with the Gradio UI

After running the cell above, you should see a link to the Gradio interface. Click on it to open the UI in a new tab. You can then interact with the AI Financial Advisor by:

1. Selecting either "Summary" or "Advice" from the dropdown menu.
2. If you selected "Advice", enter your age, lifestyle, and hobbies (optional).
3. Click the "Submit" button to generate the financial summary or personalized advice.

## Customizing the Gradio UI

You can customize the Gradio UI by modifying the `launch_gradio_ui` function. For example, you could add more input fields, change the layout, or add example inputs.

## Conclusion

In this notebook, we've created a self-contained Gradio UI for our AI Financial Advisor and integrated it with GPT for generating financial insights. This provides an interactive way for users to get financial summaries and personalized advice based on their transaction history and personal information.

You can further improve this interface by:
1. Adding data visualization components to show spending trends
2. Implementing user authentication to handle multiple users' data
3. Adding more advanced financial analysis tools

Feel free to experiment with the code and adapt it to create even more powerful and user-friendly financial advisory tools!